In [1]:
import torch
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import json
from deeplab_resnet import DeepLabv3_plus
from copy import deepcopy
import concurrent.futures
import time
%matplotlib inline

In [2]:
net = DeepLabv3_plus(n_classes=1, nInputChannels=4)
#model_name = 'models/deepgc_pascal_epoch-199_davis_finetune.pth'
model_name = '/home/ningxinL/DeepGrabCut_Davis_finetune_with_scaler/deepgc_pascal_epoch-139.pth'
checkpoint = torch.load(model_name, map_location=lambda storage, loc: storage)
pretrained_dict = {k: v for k, v in checkpoint.items() if k in net.state_dict()}
model_dict = net.state_dict()
model_dict.update(pretrained_dict)
net.load_state_dict(model_dict)
net.eval()

Constructing DeepLabv3+ model...
Number of classes: 1
Output stride: 16
Number of Input Channels: 4


DeepLabv3_plus(
  (resnet_features): ResNet(
    (conv1): Conv2d(4, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (downsample): Sequential(
     

In [3]:
with open('val_v_5.json','r') as f:
    val_file = json.load(f)

In [4]:
val_file = json.loads(val_file)

In [5]:
def compute_dismap(dismap, bbox):
    x_min, y_min, x_max, y_max = bbox[:]

    # draw bounding box
    cv2.line(dismap, (x_min, y_min), (x_max, y_min), color=1, thickness=1)
    cv2.line(dismap, (x_min, y_min), (x_min, y_max), color=1, thickness=1)
    cv2.line(dismap, (x_max, y_max), (x_max, y_min), color=1, thickness=1)
    cv2.line(dismap, (x_max, y_max), (x_min, y_max), color=1, thickness=1)

    tmp = (dismap > 0).astype(np.uint8) # mark boundary
    tmp_ = deepcopy(tmp)

    fill_mask = np.ones((tmp.shape[0] + 2, tmp.shape[1] + 2)).astype(np.uint8)
    fill_mask[1:-1, 1:-1] = tmp_
    cv2.floodFill(tmp_, fill_mask, (int((x_min + x_max) / 2), int((y_min + y_max) / 2)), 5) # fill pixel inside bounding box

    tmp_ = tmp_.astype(np.int8)
    tmp_[tmp_ == 5] = -1 # pixel inside bounding box
    tmp_[tmp_ == 0] = 1 # pixel on and outside bounding box

    tmp = (tmp == 0).astype(np.uint8)

    dismap = cv2.distanceTransform(tmp, cv2.DIST_L2, cv2.DIST_MASK_PRECISE)  # compute distance inside and outside bounding box
    dismap = tmp_ * dismap + 128

    dismap[dismap > 255] = 255
    dismap[dismap < 0] = 0

    dismap = dismap.astype(np.uint8)

    return dismap

In [6]:
def distance_map(mask,bbox):
    bounds = (0, 0, mask.shape[1] - 1, mask.shape[0] - 1)


    x_min = max(bbox[0], bounds[0])
    y_min = max(bbox[1], bounds[1])
    x_max = min(bbox[2], bounds[2])
    y_max = min(bbox[3], bounds[3])

    bbox = [x_min, y_min, x_max, y_max]

    dismap = np.zeros((mask.shape[0], mask.shape[1]))
    dismap = compute_dismap(dismap, bbox)
    return dismap

In [10]:
categorys = ['2', '1', '3', '5', '4']

In [22]:
thres = 0.75
def get_val_result(key):
    base_dir = '/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/'
    if not os.path.exists(base_dir):
        os.mkdir(base_dir)
    result_dir = os.path.join(base_dir, key)
    if not os.path.exists(result_dir):
        os.mkdir(result_dir)
    temp = val_file[key]
    for frame in temp:
        img_path = temp[frame]['img_path']
        print(img_path)
        assert(os.path.isfile(img_path))
        img = cv2.imread(img_path)
        result_shape = img.shape
        for category in categorys:
            if category in temp[frame].keys():
                mask_dir = os.path.join(result_dir, category)
                if not os.path.exists(mask_dir):
                    os.mkdir(mask_dir)
                bbox = temp[frame][category]['bbox']
                dis_map = distance_map(img, bbox)
                dis_map = cv2.resize(dis_map, (450, 450), interpolation=cv2.INTER_NEAREST)
                dis_map = dis_map[:,:, np.newaxis]
                res = cv2.resize(img, (450,450), interpolation=cv2.INTER_CUBIC)
                res = cv2.cvtColor(res, cv2.COLOR_BGR2RGB)
                res = np.concatenate((res, dis_map), axis=2)
                res = res.transpose((2,0,1))
                res = res[np.newaxis,:,:,:]
                res = torch.from_numpy(res).float()
                outputs = net.forward(res)
                outputs = outputs.to(torch.device('cpu'))
                prediction = np.transpose(outputs.data.numpy()[0, ...], (1, 2, 0))
                prediction = 1 / (1 + np.exp(-prediction))
                prediction = np.squeeze(prediction)
                prediction[prediction>thres] = 255
                prediction[prediction<=thres] = 0
                prediction = prediction.astype(np.uint8)
                prediction = cv2.resize(prediction, (result_shape[1], result_shape[0]),interpolation=cv2.INTER_NEAREST)
                result_path  = os.path.join(mask_dir, img_path.split('/')[-1].replace('.jpg', '.png'))
                print(result_path)
                cv2.imwrite(result_path, prediction)

In [23]:
keys = val_file.keys()
with concurrent.futures.ProcessPoolExecutor() as executor:
        executor.map(get_val_result, keys)

/home/ningxinL/davis2018/data/DAVIS/JPEGImages/480p/dance-twirl/00039.jpg
/home/ningxinL/davis2018/data/DAVIS/JPEGImages/480p/camel/00039.jpg
/home/ningxinL/davis2018/data/DAVIS/JPEGImages/480p/pigs/00050.jpg
/home/ningxinL/davis2018/data/DAVIS/JPEGImages/480p/parkour/00039.jpg
/home/ningxinL/davis2018/data/DAVIS/JPEGImages/480p/drift-straight/00039.jpg
/home/ningxinL/davis2018/data/DAVIS/JPEGImages/480p/blackswan/00039.jpg
/home/ningxinL/davis2018/data/DAVIS/JPEGImages/480p/bike-packing/00050.jpg
/home/ningxinL/davis2018/data/DAVIS/JPEGImages/480p/judo/00005.jpg
/home/ningxinL/davis2018/data/DAVIS/JPEGImages/480p/india/00050.jpg
/home/ningxinL/davis2018/data/DAVIS/JPEGImages/480p/libby/00039.jpg
/home/ningxinL/davis2018/data/DAVIS/JPEGImages/480p/car-roundabout/00050.jpg
/home/ningxinL/davis2018/data/DAVIS/JPEGImages/480p/drift-chicane/00050.jpg
/home/ningxinL/davis2018/data/DAVIS/JPEGImages/480p/dogs-jump/00050.jpg
/home/ningxinL/davis2018/data/DAVIS/JPEGImages/480p/cows/00039.jpg
/h

/home/ningxinL/davis2018/data/DAVIS/JPEGImages/480p/drift-straight/00038.jpg
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/car-roundabout/1/00022.png
/home/ningxinL/davis2018/data/DAVIS/JPEGImages/480p/car-roundabout/00007.jpg
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/parkour/1/00065.png
/home/ningxinL/davis2018/data/DAVIS/JPEGImages/480p/parkour/00012.jpg
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/judo/1/00032.png
/home/ningxinL/davis2018/data/DAVIS/JPEGImages/480p/judo/00023.jpg
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/bike-packing/1/00039.png
/home/ningxinL/davis2018/data/DAVIS/JPEGImages/480p/bike-packing/00032.jpg
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/india/1/00039.png
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/libby/1/00007.png
/home/ningxinL/davis2018/data/DAVIS/JPEGImages/480p/libby/00038.jpg
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/pigs/1/00039.png
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/dance-tw

/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/libby/1/00031.png
/home/ningxinL/davis2018/data/DAVIS/JPEGImages/480p/libby/00016.jpg
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/india/3/00032.png
/home/ningxinL/davis2018/data/DAVIS/JPEGImages/480p/india/00022.jpg
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/drift-straight/1/00031.png
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/bike-packing/1/00022.png
/home/ningxinL/davis2018/data/DAVIS/JPEGImages/480p/drift-straight/00016.jpg
/home/ningxinL/davis2018/data/DAVIS/JPEGImages/480p/bike-packing/00007.jpg
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/judo/1/00024.png
/home/ningxinL/davis2018/data/DAVIS/JPEGImages/480p/judo/00007.jpg
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/pigs/3/00032.png
/home/ningxinL/davis2018/data/DAVIS/JPEGImages/480p/pigs/00022.jpg
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/dance-twirl/1/00068.png
/home/ningxinL/davis2018/data/DAVIS/JPEGImages/480p/dance-twirl/0001

/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/india/2/00007.png
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/motocross-jump/1/00007.png
/home/ningxinL/davis2018/data/DAVIS/JPEGImages/480p/motocross-jump/00037.jpg
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/bike-packing/1/00057.png
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/judo/1/00011.png
/home/ningxinL/davis2018/data/DAVIS/JPEGImages/480p/judo/00015.jpg
/home/ningxinL/davis2018/data/DAVIS/JPEGImages/480p/bike-packing/00065.jpg
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/libby/1/00027.png
/home/ningxinL/davis2018/data/DAVIS/JPEGImages/480p/libby/00047.jpg
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/drift-chicane/1/00027.png
/home/ningxinL/davis2018/data/DAVIS/JPEGImages/480p/drift-chicane/00047.jpg
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/parkour/1/00037.png
/home/ningxinL/davis2018/data/DAVIS/JPEGImages/480p/parkour/00084.jpg
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/

/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/india/3/00057.png
/home/ningxinL/davis2018/data/DAVIS/JPEGImages/480p/india/00078.jpg
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/gold-fish/2/00022.png
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/bike-packing/1/00038.png
/home/ningxinL/davis2018/data/DAVIS/JPEGImages/480p/bike-packing/00036.jpg
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/motocross-jump/1/00015.png
/home/ningxinL/davis2018/data/DAVIS/JPEGImages/480p/motocross-jump/00038.jpg
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/pigs/1/00057.png
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/drift-chicane/1/00045.png
/home/ningxinL/davis2018/data/DAVIS/JPEGImages/480p/drift-chicane/00042.jpg
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/dance-twirl/1/00049.png
/home/ningxinL/davis2018/data/DAVIS/JPEGImages/480p/dance-twirl/00052.jpg
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/parkour/1/00045.png
/home/ningxinL/davis2018/data/DAVIS

...

/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/lab-coat/5/00019.png
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/gold-fish/5/00066.png
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/paragliding-launch/2/00020.png
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/soapbox/3/00079.png
/home/ningxinL/davis2018/data/DAVIS/JPEGImages/480p/soapbox/00082.jpg
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/loading/1/00004.png
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/lab-coat/4/00019.png
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/mbike-trick/1/00004.png
/home/ningxinL/davis2018/data/DAVIS/JPEGImages/480p/lab-coat/00009.jpg
/home/ningxinL/davis2018/data/DAVIS/JPEGImages/480p/mbike-trick/00043.jpg
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/gold-fish/4/00066.png
/home/ningxinL/davis2018/data/DAVIS/JPEGImages/480p/gold-fish/00025.jpg
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/paragliding-launch/1/00020.png
/home/ningxinL/DeepGrabCut_Davis_

/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/lab-coat/3/00023.png
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/gold-fish/3/00020.png
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/mbike-trick/2/00018.png
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/soapbox/1/00015.png
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/paragliding-launch/3/00017.png
/home/ningxinL/davis2018/data/DAVIS/JPEGImages/480p/paragliding-launch/00014.jpg
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/loading/2/00001.png
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/lab-coat/5/00023.png
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/mbike-trick/1/00018.png
/home/ningxinL/davis2018/data/DAVIS/JPEGImages/480p/mbike-trick/00045.jpg
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/gold-fish/5/00020.png
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/soapbox/3/00015.png
/home/ningxinL/davis2018/data/DAVIS/JPEGImages/480p/soapbox/00035.jpg
/home/ningxinL/DeepGrabCut_Dav

/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/paragliding-launch/2/00009.png
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/soapbox/3/00067.png
/home/ningxinL/davis2018/data/DAVIS/JPEGImages/480p/soapbox/00066.jpg
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/lab-coat/4/00033.png
/home/ningxinL/davis2018/data/DAVIS/JPEGImages/480p/lab-coat/00008.jpg
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/gold-fish/4/00034.png
/home/ningxinL/davis2018/data/DAVIS/JPEGImages/480p/gold-fish/00017.jpg
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/paragliding-launch/1/00009.png
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/soapbox/2/00066.png
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/lab-coat/2/00008.png
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/gold-fish/2/00017.png
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/paragliding-launch/3/00009.png
/home/ningxinL/davis2018/data/DAVIS/JPEGImages/480p/paragliding-launch/00041.jpg
/home/ningxinL/Deep

/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/soapbox/3/00017.png
/home/ningxinL/davis2018/data/DAVIS/JPEGImages/480p/soapbox/00096.jpg
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/lab-coat/2/00021.png
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/gold-fish/2/00063.png
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/paragliding-launch/1/00008.png
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/soapbox/2/00096.png
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/lab-coat/1/00021.png
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/gold-fish/1/00063.png
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/soapbox/1/00096.png
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/paragliding-launch/3/00008.png
/home/ningxinL/davis2018/data/DAVIS/JPEGImages/480p/paragliding-launch/00001.jpg
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/lab-coat/3/00021.png
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/gold-fish/3/00063.png
/home/ningxinL/DeepGrabCut

/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/gold-fish/1/00048.png
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/soapbox/3/00003.png
/home/ningxinL/davis2018/data/DAVIS/JPEGImages/480p/soapbox/00001.jpg
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/gold-fish/3/00048.png
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/soapbox/2/00001.png
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/gold-fish/5/00048.png
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/soapbox/1/00001.png
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/gold-fish/4/00048.png
/home/ningxinL/davis2018/data/DAVIS/JPEGImages/480p/gold-fish/00004.jpg
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/soapbox/3/00001.png
/home/ningxinL/davis2018/data/DAVIS/JPEGImages/480p/soapbox/00029.jpg
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/gold-fish/2/00004.png
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/soapbox/2/00029.png
/home/ningxinL/DeepGrabCut_Davis_val/result/voc_v_5/gold-f

In [21]:
for key in keys:
    get_val_result(key)

/home/ningxinL/davis2018/data/DAVIS/JPEGImages/480p/dance-twirl/00039.jpg


AssertionError: 

In [199]:
val_file['india']['35']

{'1': {'bbox': [-50, 56, 271, 529]},
 '2': {'bbox': [206, 137, 284, 396]},
 'img_path': '/home/ningxinL/davis2018/data/DAVIS/JPEGImages/480p/india/00035.jpg',
 'mask_path': '/home/ningxinL/davis2018/data/DAVIS/Annotations/480p/india/00035.png'}

In [25]:
print(keys)

dict_keys(['camel', 'pigs', 'scooter-black', 'motocross-jump', 'goat', 'gold-fish', 'car-roundabout', 'india', 'drift-straight', 'mbike-trick', 'car-shadow', 'paragliding-launch', 'dog', 'judo', 'loading', 'dance-twirl', 'dogs-jump', 'lab-coat', 'bmx-trees', 'parkour', 'kite-surf', 'blackswan', 'shooting', 'drift-chicane', 'soapbox', 'breakdance', 'libby', 'horsejump-high', 'bike-packing', 'cows'])
